***
# **Diplomová práce - výpočty**
***
### **Jméno:** Alois Primes
### **Název práce:** MODULARNÍ HORIZONTÁLNÍ KOTEL – HRSG
### **Zadání**
**Stručná charakteristika problematiky úkolu:**

Práce je zaměřena na návrh horizontálního kotle využívajícího spaliny za plynovou spalovací turbínou pro produkci páry k dalšímu energetickému využití. Kotel by měl být moderní konstrukce, zahrnující
maximální prefabrikaci transportních bloků (modulů), která je ovlivněna podmínkami dopravního profilu na místo stavby. Kotel bude řešen jako modulární, s horizontálním prouděním spalin, konvekčním
výparníkem, výhřevnými plochami ohříváku vody (ekonomizéry) a přehříváky páry s regulací teploty páry vstřikem napájecí vody, odebírané mezi dvěma ekonomizéry.

**Základní parametry kotle jsou:** parní výkon 76 t/h, tlak 3,1 MPa, výstupní teplota 370 °C. Teplota
napájecí vody kotle je 71°C. Další parametry kotle a odpadních spalin budou upřesněny v rámci
realizace práce.

**Cíle diplomové práce:**

Zpracovat tepelný výpočet kotle.
Zpracovat rozměrový návrh kotle, včetně základní výkresové dokumentace kotle a dispozičního
řešení.
Vyčíslit dosaženou účinnost kotle při vztažné teplotě 25 °C, použijte přímou metodu.
Zpracovat diagram průběhu teploty pracovního média a spalin a stanovit bilanci vstřiku.

In [168]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade tabulate
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [169]:
# Import knihoven

import pandas as pd
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e
from pyXSteam.XSteam import XSteam
from tabulate import tabulate

In [170]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W
pd.set_option("display.precision", 1)

### Definice pomocných funkcí

In [171]:
def density(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu density('prvek', teplota [K], tlak [BAR])
	Příklad rho_CO2 = density('CO2', 293.15, 5.5)
	Vypočte hustotu CO2 pro teplotu 20 °C (293.15 K) a 5.5 baru (550 000 Pa)
    JEDNOTKA: kg/m^3
	"""

	rho = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	return rho

In [172]:
def conductivity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu conductivity('prvek', teplota [K], tlak [bar])
	Příklad lambda_CO2 = conductivity('CO2', 293.15, 1.01325)
	Vypočte vodivost CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
    JEDNOTKA: W/m*K
	"""

	lambda_prvek = PropsSI('conductivity', 'T', teplota, 'P', tlak*100000, prvek)
	return lambda_prvek

In [173]:
def enthalpy(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu enthalpy('prvek', teplota [K], tlak [bar])
	Příklad h_CO2 = enthalpy('CO2', 293.15, 1.01325)
	Vypočte entalpii CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg
	"""

	h_prvek = PropsSI('H', 'T', teplota, 'P', tlak*100000, prvek)
	return h_prvek

In [174]:
def viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu viscosity('prvek', teplota [K], tlak [bar])
	Příklad eta_CO2 = viscosity('CO2', 293.15, 1.01325)
	Vypočte viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: Pa.s
	"""

	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	return eta_prvek

In [175]:
def cp(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu cp('prvek', teplota [K], tlak [bar])
	Příklad cp_CO2 = cp('CO2', 293.15, 1.01325)
	Vypočte měrnou tepelnou kapacitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg*K
	"""

	cp_prvek = PropsSI('C', 'T', teplota, 'P', tlak*100000, prvek)
	return cp_prvek

In [176]:
def kin_viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu kin_viscosity('prvek', teplota [K], tlak [bar])
	Příklad nu_CO2 = kin_viscosity('CO2', 293.15, 1.01325)
	Vypočte kinematickou viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: m^2/s
	"""
	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	rho_prvek = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	nu_prvek = eta_prvek/rho_prvek
	return nu_prvek

***
## Základní parametry kotle
***

In [224]:
%%render params 3
m_p = (76*1000)/3600        #kg/s
p_out = 3.1                 #MPa
t_out = 370                 #°C
t_nv = 71                   #°C
m_vst = 0.36                #kg/s
t_vst = 217                 #°C

<IPython.core.display.Latex object>

***
## Parametry spalin ze spalovací turbíny
***

In [178]:
R = 8.314462618 #J/kg/mol
T = 273.15 #K

#### koeficienty polynomů pro výpočet entalpií spalin

In [179]:
# N2
a_1_N2 = 0.3694063e+01
a_2_N2 = -0.1334139e-02
a_3_N2 = 0.2652067e-05
a_4_N2 = -0.9775312e-09
a_5_N2 = -0.9983837e-13
a_6_N2 = -0.9758881e+03

# O2
a_1_O2 = 0.3721461e+01
a_2_O2 = -0.2518398e-02
a_3_O2 = 0.8589429e-05
a_4_O2 = -0.8305377e-08
a_5_O2 = 0.2710013e-11
a_6_O2 = -0.9701834e+03

# CO2
a_1_CO2 = 0.2171539e+01
a_2_CO2 = 0.1038499e-01
a_3_CO2 = -0.1074106e-04
a_4_CO2 = 0.6350127e-08
a_5_CO2 = -0.1629149e-11
a_6_CO2 = -0.9159472e+03

#### Objemové složení spalin, hmotnostní průtok a teplota na vstupu do hořáku

In [180]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

#### Molární hmotnosti spalin

In [181]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134e-03      #kg/kmol
M_O2 = 31.9988e-03      #kg/kmol
M_H2O = 18.0152e-03     #kg/kmol
M_CO2 = 44.0095e-03     #kg/kmol
M_Ar = 39.9480e-03      #kg/kmol

<IPython.core.display.Latex object>

#### Hustota složek spalin (z podkladů)

In [182]:
%%render params
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3
rho_sp = x_N2*rho_N2 + x_O2*rho_O2 + x_H2O*rho_H2O + x_CO2*rho_CO2 + x_Ar*rho_Ar #kg/m3

<IPython.core.display.Latex object>

#### Objemový průtok spalin

In [226]:
%%render
V_sp = m_sp/rho_sp #m3n/s

<IPython.core.display.Latex object>

In [184]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

#### Entalpie spalin (N,O,CO2 z materiálů, H2O a Ar od Ing. Křeminského)

In [185]:
# Teploty pro tabulku entalpií spalin
t_100 = 100+T
t_200 = 200+T
t_300 = 300+T
t_400 = 400+T
t_500 = 500+T
t_600 = 600+T

#### t = 100 °C

In [186]:
%%render params

I_N2_100 = (a_1_N2 + a_2_N2*t_100/2 + a_3_N2*t_100**2/3 + a_4_N2*t_100**3/4 + a_5_N2*t_100**4/5 + a_6_N2/t_100)*((R*t_100)/M_N2)/1000 #kJ/m3n
I_O2_100 = (a_1_O2 + a_2_O2*t_100/2 + a_3_O2*t_100**2/3 + a_4_O2*t_100**3/4 + a_5_O2*t_100**4/5 + a_6_O2/t_100)*((R*t_100)/M_O2)/1000 #kJ/m3n
I_H2O_100 = 150 #kJ/m3n
I_CO2_100 = (a_1_CO2 + a_2_CO2*t_100/2 + a_3_CO2*t_100**2/3 + a_4_CO2*t_100**3/4 + a_5_CO2*t_100**4/5 + a_6_CO2/t_100)*((R*t_100)/M_CO2)/1000 #kJ/m3n
I_Ar_100 = 93 #kJ/m3n

<IPython.core.display.Latex object>

In [231]:
%%render
# Vzorový výpočet
I_sp_100 = x_N2*I_N2_100 + x_O2*I_O2_100 + x_H2O*I_H2O_100 + x_CO2*I_CO2_100 + x_Ar*I_Ar_100 #kJ/m3n

<IPython.core.display.Latex object>

#### t = 200 °C

In [188]:
%%render params

I_N2_200 = (a_1_N2 + a_2_N2*t_200/2 + a_3_N2*t_200**2/3 + a_4_N2*t_200**3/4 + a_5_N2*t_200**4/5 + a_6_N2/t_200)*((R*t_200)/M_N2)/1000 #kJ/m3n
I_O2_200 = (a_1_O2 + a_2_O2*t_200/2 + a_3_O2*t_200**2/3 + a_4_O2*t_200**3/4 + a_5_O2*t_200**4/5 + a_6_O2/t_200)*((R*t_200)/M_O2)/1000 #kJ/m3n
I_H2O_200 = 304 #kJ/m3n
I_CO2_200 = (a_1_CO2 + a_2_CO2*t_200/2 + a_3_CO2*t_200**2/3 + a_4_CO2*t_200**3/4 + a_5_CO2*t_200**4/5 + a_6_CO2/t_200)*((R*t_200)/M_CO2)/1000 #kJ/m3n
I_Ar_200 = 186 #kJ/m3n

I_sp_200 = x_N2*I_N2_200 + x_O2*I_O2_200 + x_H2O*I_H2O_200 + x_CO2*I_CO2_200 + x_Ar*I_Ar_200 #kJ/m3n

<IPython.core.display.Latex object>

#### t = 300 °C

In [189]:
%%render params

I_N2_300 = (a_1_N2 + a_2_N2*t_300/2 + a_3_N2*t_300**2/3 + a_4_N2*t_300**3/4 + a_5_N2*t_300**4/5 + a_6_N2/t_300)*((R*t_300)/M_N2)/1000 #kJ/m3n
I_O2_300 = (a_1_O2 + a_2_O2*t_300/2 + a_3_O2*t_300**2/3 + a_4_O2*t_300**3/4 + a_5_O2*t_300**4/5 + a_6_O2/t_300)*((R*t_300)/M_O2)/1000 #kJ/m3n
I_H2O_300 = 463 #kJ/m3n
I_CO2_300 = (a_1_CO2 + a_2_CO2*t_300/2 + a_3_CO2*t_300**2/3 + a_4_CO2*t_300**3/4 + a_5_CO2*t_300**4/5 + a_6_CO2/t_300)*((R*t_300)/M_CO2)/1000 #kJ/m3n
I_Ar_300 = 278 #kJ/m3n

I_sp_300 = x_N2*I_N2_300 + x_O2*I_O2_300 + x_H2O*I_H2O_300 + x_CO2*I_CO2_300 + x_Ar*I_Ar_300 #kJ/m3n

<IPython.core.display.Latex object>

#### t = 400 °C

In [190]:
%%render params

I_N2_400 = (a_1_N2 + a_2_N2*t_400/2 + a_3_N2*t_400**2/3 + a_4_N2*t_400**3/4 + a_5_N2*t_400**4/5 + a_6_N2/t_400)*((R*t_400)/M_N2)/1000 #kJ/m3n
I_O2_400 = (a_1_O2 + a_2_O2*t_400/2 + a_3_O2*t_400**2/3 + a_4_O2*t_400**3/4 + a_5_O2*t_400**4/5 + a_6_O2/t_400)*((R*t_400)/M_O2)/1000 #kJ/m3n
I_H2O_400 = 626 #kJ/m3n
I_CO2_400 = (a_1_CO2 + a_2_CO2*t_400/2 + a_3_CO2*t_400**2/3 + a_4_CO2*t_400**3/4 + a_5_CO2*t_400**4/5 + a_6_CO2/t_400)*((R*t_400)/M_CO2)/1000 #kJ/m3n
I_Ar_400 = 372 #kJ/m3n

I_sp_400 = x_N2*I_N2_400 + x_O2*I_O2_400 + x_H2O*I_H2O_400 + x_CO2*I_CO2_400 + x_Ar*I_Ar_400 #kJ/m3n

<IPython.core.display.Latex object>

#### t = 500 °C

In [191]:
%%render params

I_N2_500 = (a_1_N2 + a_2_N2*t_500/2 + a_3_N2*t_500**2/3 + a_4_N2*t_500**3/4 + a_5_N2*t_500**4/5 + a_6_N2/t_500)*((R*t_500)/M_N2)/1000 #kJ/m3n
I_O2_500 = (a_1_O2 + a_2_O2*t_500/2 + a_3_O2*t_500**2/3 + a_4_O2*t_500**3/4 + a_5_O2*t_500**4/5 + a_6_O2/t_500)*((R*t_500)/M_O2)/1000 #kJ/m3n
I_H2O_500 = 795 #kJ/m3n
I_CO2_500 = (a_1_CO2 + a_2_CO2*t_500/2 + a_3_CO2*t_500**2/3 + a_4_CO2*t_500**3/4 + a_5_CO2*t_500**4/5 + a_6_CO2/t_500)*((R*t_500)/M_CO2)/1000 #kJ/m3n
I_Ar_500 = 456 #kJ/m3n

I_sp_500 = x_N2*I_N2_500 + x_O2*I_O2_500 + x_H2O*I_H2O_500 + x_CO2*I_CO2_500 + x_Ar*I_Ar_500 #kJ/m3n

<IPython.core.display.Latex object>

#### t = 600 °C

In [192]:
%%render params

I_N2_600 = (a_1_N2 + a_2_N2*t_600/2 + a_3_N2*t_600**2/3 + a_4_N2*t_600**3/4 + a_5_N2*t_600**4/5 + a_6_N2/t_600)*((R*t_600)/M_N2)/1000 #kJ/m3n
I_O2_600 = (a_1_O2 + a_2_O2*t_600/2 + a_3_O2*t_600**2/3 + a_4_O2*t_600**3/4 + a_5_O2*t_600**4/5 + a_6_O2/t_600)*((R*t_600)/M_O2)/1000 #kJ/m3n
I_H2O_600 = 969 #kJ/m3n
I_CO2_600 = (a_1_CO2 + a_2_CO2*t_600/2 + a_3_CO2*t_600**2/3 + a_4_CO2*t_600**3/4 + a_5_CO2*t_600**4/5 + a_6_CO2/t_600)*((R*t_600)/M_CO2)/1000 #kJ/m3n
I_Ar_600 = 557 #kJ/m3n

I_sp_600 = x_N2*I_N2_600 + x_O2*I_O2_600 + x_H2O*I_H2O_600 + x_CO2*I_CO2_600 + x_Ar*I_Ar_600 #kJ/m3n

<IPython.core.display.Latex object>

#### Tabulka - Entalpie jednotlivych prvku a spalin pro dane teploty

In [193]:
spaliny_tabulka = pd.DataFrame(
    {
        "T [°C]": (t_100-T, t_200-T, t_300-T, t_400-T, t_500-T, t_600-T),
        "N2 [kJ/m3n]": (i_N2_100, i_N2_200, i_N2_300, i_N2_400, i_N2_500, i_N2_600),
        "O2 [kJ/m3n]": (i_O2_100, i_O2_200, i_O2_300, i_O2_400, i_O2_500, i_O2_600),
        "H2O [kJ/m3n]": (i_H2O_100, i_H2O_200, i_H2O_300, i_H2O_400, i_H2O_500, i_H2O_600),
        "CO2 [kJ/m3n]": (i_CO2_100, i_CO2_200, i_CO2_300, i_CO2_400, i_CO2_500, i_CO2_600),
        "Ar [kJ/m3n]": (i_Ar_100, i_Ar_200, i_Ar_300, i_Ar_400, i_Ar_500, i_Ar_600),
        "Spaliny [kJ/m3n]": (i_sp_100, i_sp_200, i_sp_300, i_sp_400, i_sp_500, i_sp_600),
    }
)
spaliny_tabulka.set_index('T [°C]', inplace=True)

In [229]:
spaliny_tabulka

# print(spaliny_tabulka.to_latex())

,N2 [kJ/m3n],O2 [kJ/m3n],H2O [kJ/m3n],CO2 [kJ/m3n],Ar [kJ/m3n],Spaliny [kJ/m3n]
T [°C],,,,,,
100.0,104.1,92.4,150,86.9,93,105.5
200.0,208.8,187.3,304,182.6,186,212.5
300.0,314.9,285.2,463,285.6,278,321.6
400.0,423.0,386.2,626,394.4,372,433.0
500.0,533.4,489.9,795,508.1,456,547.1
600.0,646.2,595.9,969,625.9,557,664.1


***
### Spaliny - body
***

### <u>Bod A (Vstup spalin - za hořákem)

In [233]:
%%render
t_A = t_sp #°C
I_A_sp = (I_sp_600-I_sp_500)/(600-500) * (t_A-500) + I_sp_500 #kJ/m3n

<IPython.core.display.Latex object>

### Bod B (za druhým přehřívákem)

In [235]:
%%render
t_B = 549 #°C
I_B_sp = (I_sp_600-I_sp_500)/(600-500) * (t_B-500) + I_sp_500 #kJ/m3n

<IPython.core.display.Latex object>

### Bod C (za prvním přehřívákem)

In [237]:
%%render
t_C = 507 #°C
I_C_sp = (I_sp_600-I_sp_500)/(600-500) * (t_C-500) + I_sp_500 #kJ/m3n

<IPython.core.display.Latex object>

### Bod D (za druhým výparníkem)

In [239]:
%%render
t_D = 366 #°C
I_D_sp = (I_sp_400-I_sp_300)/(400-300) * (t_D-300) + I_sp_300 #kJ/m3n

<IPython.core.display.Latex object>

### Bod E (za prvním výparníkem)

In [241]:
%%render
t_E = 254 #°C
I_E_sp = (I_sp_300-I_sp_200)/(300-200) * (t_E-200) + I_sp_200 #kJ/m3n

<IPython.core.display.Latex object>

### Bod F (za druhým ekonomizérem)

In [243]:
%%render
t_F = 241 #°C
I_F_sp = (I_sp_300-I_sp_200)/(300-200) * (t_F-200) + I_sp_200 #kJ/m3n

<IPython.core.display.Latex object>

### Bod G (za prvním ekonomizérem, výstup spalin)

In [245]:
%%render
t_G = 145 #°C
I_G_sp = (I_sp_200-I_sp_100)/(200-100) * (t_G-100) + I_sp_100 #kJ/m3n

<IPython.core.display.Latex object>

***
## Výpočty ze strany vody a vodní páry
***

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [202]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
i_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)    #m3/kg
x_1 = steam.x_ph(p_1*10,i_1)    #-

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [203]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
i_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg
x_3 = steam.x_ph(p_3*10,i_3)    #-

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [204]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
i_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg
x_4 = steam.x_ph(p_4*10,i_4)    #-

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [205]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
i_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg
x_5 = steam.x_ph(p_5*10,i_5)    #-

<IPython.core.display.Latex object>

#### Bod 6 (Za posledním ekonomizérem, před vstupem do výparníku)

In [206]:
%%render params
p_6 = 3.68                      #MPa
t_6 = 236                       #°C
i_6 = steam.h_pt(p_6*10, t_6)   #kJ/kg
v_6 = steam.v_pt(p_6*10, t_6)   #m3/kg
x_6 = steam.x_ph(p_6*10,i_6)    #-

<IPython.core.display.Latex object>

#### Bod 7 (mezi ekonomizéry)

In [207]:
%%render params
p_7 = 3.74                      #MPa
t_7 = 217                       #°C
i_7 = steam.h_pt(p_7*10, t_7)   #kJ/kg
v_7 = steam.v_pt(p_7*10, t_7)   #m3/kg
x_7 = steam.x_ph(p_7*10,i_7)    #-


<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku)

In [208]:
%%render params
p_2 = p_3                           #MPa
i_2 = (20.74*i_3 + 0.36*i_7)/21.10  #kJ/kg
t_2 = steam.t_ph(p_2*10, i_2)       #°C
v_2 = steam.v_ph(p_2*10, i_2)       #m3/kg
x_2 = steam.x_ph(p_2*10,i_2)        #-

<IPython.core.display.Latex object>

#### Bod 8 (Před začátkem ekonomizéru, vstup napájecí vody)

In [209]:
%%render params
t_8 = 71                         #°C
p_8 = 4                          #MPa (NEVIM)
i_8 = steam.h_pt(p_8*10,t_8)     #kJ/kg
v_8 = steam.v_pt(p_8*10,t_8)     #m3/kg
x_8 = steam.x_ph(p_8*10,i_8)    #-

<IPython.core.display.Latex object>

### Tabulka bodová pro vodu a páru

In [210]:
bodova_tabulka = pd.DataFrame(
    {
        "T [°C]": (t_1, t_2, t_3, t_4, t_5, t_6, t_7, t_8),
        "p [MPa]": (p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8),
        "i [kJ/kg]": (i_1, i_2, i_3, i_4, i_5, i_6, i_7, i_8),
        "v [m3/kg]": (v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8),
        "x [-]": (x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8),
    },
    index = ['1','2','3','4','5','6','7','8']
)

In [211]:
bodova_tabulka

# print(bodova_tabulka.to_latex())

,T [°C],p [MPa],i [kJ/kg],v [m3/kg],x [-]
1,370.0,3.1,3160.7,9.1e-02,1.0
2,332.7,3.3,3066.8,7.8e-02,1.0
3,348.0,3.3,3103.9,8.1e-02,1.0
4,240.6,3.4,2803.0,5.9e-02,1.0
5,240.6,3.4,1040.2,1.2e-03,0.0
6,236.0,3.7,1018.6,1.2e-03,0.0
7,217.0,3.7,930.3,1.2e-03,0.0
8,71.0,4.0,300.4,1.0e-03,0.0


***
## Výkon kotle a jednotlivých teplosměnných ploch
***

### Maximální využitelný výkon

In [248]:
%%render
Q_N = (I_A_sp*V_sp)/1000 #MW

<IPython.core.display.Latex object>

### Tepelné ztráty (konvekce a sálání)

In [249]:
%%render
C = 0.0113
Q_RC = Q_N**0.7*C #MW

<IPython.core.display.Latex object>

#### Poměrná ztráta sáláním a konvekcí

In [251]:
%%render
z_s = Q_RC/Q_N #-

<IPython.core.display.Latex object>

### Tepelny vykon odevzdany spalinami mezi body A-E

In [253]:
%%render
Q_AE = (I_A_sp - I_E_sp)*V_sp #kW

<IPython.core.display.Latex object>

### Tepelny vykon prijaty vodou mezi body 1-6

In [255]:
%%render
Q_16 = Q_AE*(1-z_s) #kW

<IPython.core.display.Latex object>

### Predane tepelne toky v jednotlivych teplosmennych plochach

#### Prehrivak 2

In [257]:
%%render
Q_P2 = m_p*(i_1-i_2) #kW

<IPython.core.display.Latex object>

#### Prehrivak 1

In [259]:
%%render
Q_P1 = (m_p-m_vst)*(i_3-i_4) #kW

<IPython.core.display.Latex object>

#### Vyparnik 1 + 2

In [261]:
%%render
Q_V = (m_p-m_vst)*(i_4-i_5) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 2

In [263]:
%%render
Q_EKO_2 = (m_p-m_vst)*(i_6-i_7) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 1

In [265]:
%%render
Q_EKO_1 = m_p*(i_7-i_8) #kW

<IPython.core.display.Latex object>

***
SPATNE, NEVYCHAZI KONTROLA - JSOU SPATNE ENTALPIE (tabulka entalpii spalin ze semestralky ma vyrazne odlisne hodnoty = pravdepodobna pricina nerovnosti!)
***

#### Kontrolni vypocet

In [222]:
%%render
Q_16
Q_kontrolni = Q_V + Q_P1 + Q_P2

<IPython.core.display.Latex object>